In [41]:
import logging
import pandas as pd
from sodapy import Socrata
from datetime import datetime
import pymongo
import expiringdict
import utils
logger = logging.Logger(__name__)
utils.setup_logger(logger, 'db.log')

client = Socrata("data.lacity.org", None)
results = client.get("yru6-6re4",where="arst_date > '2019-11-30T00:00:00.000'") # wait to be confirmed
df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
#df['temp'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time())
df['age'] = pd.to_numeric(df['age'])
clientm = pymongo.MongoClient()
db = clientm.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
            update_count += 1
    logger.info("rows={}, update={}, ".format(df.shape[0], update_count) +
                "insert={}".format(df.shape[0]-update_count))

2019-12-12 23:47:18,789 [<module>]: rows=990, update=1, insert=989
2019-12-12 23:47:18,794 [<module>]: rows=990, update=2, insert=988
2019-12-12 23:47:18,797 [<module>]: rows=990, update=3, insert=987
2019-12-12 23:47:18,801 [<module>]: rows=990, update=4, insert=986
2019-12-12 23:47:18,803 [<module>]: rows=990, update=5, insert=985
2019-12-12 23:47:18,805 [<module>]: rows=990, update=6, insert=984
2019-12-12 23:47:18,807 [<module>]: rows=990, update=7, insert=983
2019-12-12 23:47:18,809 [<module>]: rows=990, update=8, insert=982
2019-12-12 23:47:18,811 [<module>]: rows=990, update=9, insert=981
2019-12-12 23:47:18,813 [<module>]: rows=990, update=10, insert=980
2019-12-12 23:47:18,815 [<module>]: rows=990, update=11, insert=979
2019-12-12 23:47:18,817 [<module>]: rows=990, update=12, insert=978
2019-12-12 23:47:18,818 [<module>]: rows=990, update=13, insert=977
2019-12-12 23:47:18,820 [<module>]: rows=990, update=14, insert=976
2019-12-12 23:47:18,822 [<module>]: rows=990, update=15, 

2019-12-12 23:47:19,050 [<module>]: rows=990, update=122, insert=868
2019-12-12 23:47:19,052 [<module>]: rows=990, update=123, insert=867
2019-12-12 23:47:19,054 [<module>]: rows=990, update=124, insert=866
2019-12-12 23:47:19,056 [<module>]: rows=990, update=125, insert=865
2019-12-12 23:47:19,058 [<module>]: rows=990, update=126, insert=864
2019-12-12 23:47:19,060 [<module>]: rows=990, update=127, insert=863
2019-12-12 23:47:19,062 [<module>]: rows=990, update=128, insert=862
2019-12-12 23:47:19,064 [<module>]: rows=990, update=129, insert=861
2019-12-12 23:47:19,066 [<module>]: rows=990, update=130, insert=860
2019-12-12 23:47:19,068 [<module>]: rows=990, update=131, insert=859
2019-12-12 23:47:19,070 [<module>]: rows=990, update=132, insert=858
2019-12-12 23:47:19,072 [<module>]: rows=990, update=133, insert=857
2019-12-12 23:47:19,073 [<module>]: rows=990, update=134, insert=856
2019-12-12 23:47:19,076 [<module>]: rows=990, update=135, insert=855
2019-12-12 23:47:19,079 [<module>]

2019-12-12 23:47:19,294 [<module>]: rows=990, update=241, insert=749
2019-12-12 23:47:19,296 [<module>]: rows=990, update=242, insert=748
2019-12-12 23:47:19,297 [<module>]: rows=990, update=243, insert=747
2019-12-12 23:47:19,299 [<module>]: rows=990, update=244, insert=746
2019-12-12 23:47:19,301 [<module>]: rows=990, update=245, insert=745
2019-12-12 23:47:19,304 [<module>]: rows=990, update=246, insert=744
2019-12-12 23:47:19,307 [<module>]: rows=990, update=247, insert=743
2019-12-12 23:47:19,311 [<module>]: rows=990, update=248, insert=742
2019-12-12 23:47:19,314 [<module>]: rows=990, update=249, insert=741
2019-12-12 23:47:19,316 [<module>]: rows=990, update=250, insert=740
2019-12-12 23:47:19,320 [<module>]: rows=990, update=251, insert=739
2019-12-12 23:47:19,323 [<module>]: rows=990, update=252, insert=738
2019-12-12 23:47:19,325 [<module>]: rows=990, update=253, insert=737
2019-12-12 23:47:19,328 [<module>]: rows=990, update=254, insert=736
2019-12-12 23:47:19,331 [<module>]

2019-12-12 23:47:19,576 [<module>]: rows=990, update=360, insert=630
2019-12-12 23:47:19,577 [<module>]: rows=990, update=361, insert=629
2019-12-12 23:47:19,579 [<module>]: rows=990, update=362, insert=628
2019-12-12 23:47:19,582 [<module>]: rows=990, update=363, insert=627
2019-12-12 23:47:19,583 [<module>]: rows=990, update=364, insert=626
2019-12-12 23:47:19,585 [<module>]: rows=990, update=365, insert=625
2019-12-12 23:47:19,587 [<module>]: rows=990, update=366, insert=624
2019-12-12 23:47:19,589 [<module>]: rows=990, update=367, insert=623
2019-12-12 23:47:19,591 [<module>]: rows=990, update=368, insert=622
2019-12-12 23:47:19,593 [<module>]: rows=990, update=369, insert=621
2019-12-12 23:47:19,595 [<module>]: rows=990, update=370, insert=620
2019-12-12 23:47:19,597 [<module>]: rows=990, update=371, insert=619
2019-12-12 23:47:19,599 [<module>]: rows=990, update=372, insert=618
2019-12-12 23:47:19,601 [<module>]: rows=990, update=373, insert=617
2019-12-12 23:47:19,604 [<module>]

2019-12-12 23:47:19,857 [<module>]: rows=990, update=479, insert=511
2019-12-12 23:47:19,861 [<module>]: rows=990, update=480, insert=510
2019-12-12 23:47:19,865 [<module>]: rows=990, update=481, insert=509
2019-12-12 23:47:19,868 [<module>]: rows=990, update=482, insert=508
2019-12-12 23:47:19,871 [<module>]: rows=990, update=483, insert=507
2019-12-12 23:47:19,874 [<module>]: rows=990, update=484, insert=506
2019-12-12 23:47:19,876 [<module>]: rows=990, update=485, insert=505
2019-12-12 23:47:19,880 [<module>]: rows=990, update=486, insert=504
2019-12-12 23:47:19,883 [<module>]: rows=990, update=487, insert=503
2019-12-12 23:47:19,886 [<module>]: rows=990, update=488, insert=502
2019-12-12 23:47:19,889 [<module>]: rows=990, update=489, insert=501
2019-12-12 23:47:19,892 [<module>]: rows=990, update=490, insert=500
2019-12-12 23:47:19,895 [<module>]: rows=990, update=491, insert=499
2019-12-12 23:47:19,898 [<module>]: rows=990, update=492, insert=498
2019-12-12 23:47:19,902 [<module>]

2019-12-12 23:47:20,116 [<module>]: rows=990, update=598, insert=392
2019-12-12 23:47:20,119 [<module>]: rows=990, update=599, insert=391
2019-12-12 23:47:20,121 [<module>]: rows=990, update=600, insert=390
2019-12-12 23:47:20,123 [<module>]: rows=990, update=601, insert=389
2019-12-12 23:47:20,125 [<module>]: rows=990, update=602, insert=388
2019-12-12 23:47:20,126 [<module>]: rows=990, update=603, insert=387
2019-12-12 23:47:20,128 [<module>]: rows=990, update=604, insert=386
2019-12-12 23:47:20,130 [<module>]: rows=990, update=605, insert=385
2019-12-12 23:47:20,132 [<module>]: rows=990, update=606, insert=384
2019-12-12 23:47:20,134 [<module>]: rows=990, update=607, insert=383
2019-12-12 23:47:20,137 [<module>]: rows=990, update=608, insert=382
2019-12-12 23:47:20,139 [<module>]: rows=990, update=609, insert=381
2019-12-12 23:47:20,141 [<module>]: rows=990, update=610, insert=380
2019-12-12 23:47:20,143 [<module>]: rows=990, update=611, insert=379
2019-12-12 23:47:20,145 [<module>]

2019-12-12 23:47:20,381 [<module>]: rows=990, update=717, insert=273
2019-12-12 23:47:20,384 [<module>]: rows=990, update=718, insert=272
2019-12-12 23:47:20,386 [<module>]: rows=990, update=719, insert=271
2019-12-12 23:47:20,389 [<module>]: rows=990, update=720, insert=270
2019-12-12 23:47:20,391 [<module>]: rows=990, update=721, insert=269
2019-12-12 23:47:20,393 [<module>]: rows=990, update=722, insert=268
2019-12-12 23:47:20,395 [<module>]: rows=990, update=723, insert=267
2019-12-12 23:47:20,397 [<module>]: rows=990, update=724, insert=266
2019-12-12 23:47:20,400 [<module>]: rows=990, update=725, insert=265
2019-12-12 23:47:20,402 [<module>]: rows=990, update=726, insert=264
2019-12-12 23:47:20,404 [<module>]: rows=990, update=727, insert=263
2019-12-12 23:47:20,406 [<module>]: rows=990, update=728, insert=262
2019-12-12 23:47:20,408 [<module>]: rows=990, update=729, insert=261
2019-12-12 23:47:20,410 [<module>]: rows=990, update=730, insert=260
2019-12-12 23:47:20,411 [<module>]

2019-12-12 23:47:20,683 [<module>]: rows=990, update=836, insert=154
2019-12-12 23:47:20,686 [<module>]: rows=990, update=837, insert=153
2019-12-12 23:47:20,688 [<module>]: rows=990, update=838, insert=152
2019-12-12 23:47:20,691 [<module>]: rows=990, update=839, insert=151
2019-12-12 23:47:20,694 [<module>]: rows=990, update=840, insert=150
2019-12-12 23:47:20,697 [<module>]: rows=990, update=841, insert=149
2019-12-12 23:47:20,700 [<module>]: rows=990, update=842, insert=148
2019-12-12 23:47:20,703 [<module>]: rows=990, update=843, insert=147
2019-12-12 23:47:20,706 [<module>]: rows=990, update=844, insert=146
2019-12-12 23:47:20,708 [<module>]: rows=990, update=845, insert=145
2019-12-12 23:47:20,711 [<module>]: rows=990, update=846, insert=144
2019-12-12 23:47:20,714 [<module>]: rows=990, update=847, insert=143
2019-12-12 23:47:20,717 [<module>]: rows=990, update=848, insert=142
2019-12-12 23:47:20,719 [<module>]: rows=990, update=849, insert=141
2019-12-12 23:47:20,722 [<module>]

2019-12-12 23:47:20,970 [<module>]: rows=990, update=956, insert=34
2019-12-12 23:47:20,972 [<module>]: rows=990, update=957, insert=33
2019-12-12 23:47:20,976 [<module>]: rows=990, update=958, insert=32
2019-12-12 23:47:20,979 [<module>]: rows=990, update=959, insert=31
2019-12-12 23:47:20,981 [<module>]: rows=990, update=960, insert=30
2019-12-12 23:47:20,985 [<module>]: rows=990, update=961, insert=29
2019-12-12 23:47:20,987 [<module>]: rows=990, update=962, insert=28
2019-12-12 23:47:20,990 [<module>]: rows=990, update=963, insert=27
2019-12-12 23:47:20,993 [<module>]: rows=990, update=964, insert=26
2019-12-12 23:47:20,996 [<module>]: rows=990, update=965, insert=25
2019-12-12 23:47:20,999 [<module>]: rows=990, update=966, insert=24
2019-12-12 23:47:21,002 [<module>]: rows=990, update=967, insert=23
2019-12-12 23:47:21,005 [<module>]: rows=990, update=968, insert=22
2019-12-12 23:47:21,009 [<module>]: rows=990, update=969, insert=21
2019-12-12 23:47:21,011 [<module>]: rows=990, up

In [2]:
db = clientm.get_database("crime")
collection = db.get_collection("crime")
ret = list(collection.find())
logger.info(str(len(ret)) + ' documents read from the db')
ret

2019-12-12 22:49:54,596 [<module>]: 275 documents read from the db


[{'_id': ObjectId('5df2bd2fbeb3d39ade37441b'),
  'rpt_id': '190819078',
  'arst_date': datetime.datetime(2019, 12, 1, 0, 0),
  'time': '1300',
  'area': '08',
  'area_desc': 'West LA',
  'rd': '0817',
  'age': '30',
  'sex_cd': 'M',
  'descent_cd': 'O',
  'chrg_grp_cd': nan,
  'grp_description': nan,
  'arst_typ_cd': 'M',
  'charge': '308(A)1PC',
  'chrg_desc': nan,
  'location': '900    WESTWOOD                     BL',
  'crsst': nan,
  'location_1': {'latitude': '34.0637',
   'longitude': '-118.4453',
   'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}},
 {'_id': ObjectId('5df2bd2fbeb3d39ade3744d4'),
  'rpt_id': '5812713',
  'arst_date': datetime.datetime(2019, 12, 2, 0, 0),
  'time': '0930',
  'area': '15',
  'area_desc': 'N Hollywood',
  'rd': '1523',
  'age': '54',
  'sex_cd': 'M',
  'descent_cd': 'O',
  'chrg_grp_cd': '04',
  'grp_description': 'Aggravated Assault',
  'arst_typ_cd': 'F',
  'charge': '245(A)(1)PC',
  'chrg_desc': 'ADW, NOT FIREARM, W/GBI',


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 17 columns):
rpt_id             990 non-null object
arst_date          990 non-null datetime64[ns]
time               990 non-null object
area               990 non-null object
area_desc          990 non-null object
rd                 990 non-null object
age                990 non-null int64
sex_cd             990 non-null object
descent_cd         990 non-null object
chrg_grp_cd        930 non-null object
grp_description    930 non-null object
arst_typ_cd        990 non-null object
charge             990 non-null object
chrg_desc          937 non-null object
location           990 non-null object
crsst              485 non-null object
location_1         990 non-null object
dtypes: datetime64[ns](1), int64(1), object(15)
memory usage: 131.6+ KB


In [38]:
df['temp'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)

In [40]:
df['temp']

0       3
1       3
2      15
3       2
4      20
       ..
985     2
986    12
987     8
988     6
989     3
Name: temp, Length: 990, dtype: int64